<a href="https://colab.research.google.com/github/IAMDSVSSANGRAL/UNSUPERVISEDALGO/blob/main/Application_of_KNNRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies_df = pd.read_csv('movies.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
rating_df=pd.read_csv('ratings.csv',usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [ ]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [ ]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
df = pd.merge(rating_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [ ]:
df.shape

(100836, 4)

In [ ]:
#Remove the rows with NaN values
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])

In [ ]:
combine_movie_rating.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [ ]:
combine_movie_rating.shape

(100836, 4)

In [ ]:
movie_ratingCount = (combine_movie_rating.groupby(by = ['title'])['rating'].count().reset_index().
                    rename(columns = {'rating': 'totalRatingCount'})[['title', 'totalRatingCount']])


movie_ratingCount.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [ ]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')

rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print(movie_ratingCount['totalRatingCount'].describe())

count   9719.0000
mean      10.3751
std       22.4062
min        1.0000
25%        1.0000
50%        3.0000
75%        9.0000
max      329.0000
Name: totalRatingCount, dtype: float64


In [ ]:
popularity_threshold = 50

rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')

rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0000,Toy Story (1995),215
1,5,1,4.0000,Toy Story (1995),215
2,7,1,4.5000,Toy Story (1995),215
3,15,1,2.5000,Toy Story (1995),215
4,17,1,4.5000,Toy Story (1995),215


In [ ]:
rating_popular_movie.shape

(41362, 5)

In [ ]:
 ## First lets create a Pivot matrix
movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)

movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,3.0000,0.0000,5.0000,0.0000,0.0000,0.0000,0.0000,0.0000
12 Angry Men (1957),0.0000,0.0000,0.0000,5.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,5.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2001: A Space Odyssey (1968),0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,5.0000,0.0000,0.0000,5.0000,0.0000,3.0000,0.0000,4.5000
28 Days Later (2002),0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.5000,0.0000,5.0000
300 (2007),0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.0000,...,0.0000,0.0000,0.0000,0.0000,3.0000,0.0000,0.0000,5.0000,0.0000,4.0000


In [ ]:
movie_features_df.values

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 3. , 0. , 4.5],
       ...,
       [5. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 3. , 0. , ..., 0. , 0. , 3.5],
       [0. , 0. , 0. , ..., 3. , 0. , 4. ]], dtype=float32)

In [ ]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

In [ ]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
query_index = np.random.choice(movie_features_df.shape[0])

print(query_index)

354


In [ ]:
#predicting the 6 nearest neighbours
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
print(distances)
print(indices)

[[0.         0.3822636  0.42951941 0.4490357  0.4510743  0.45461637]]
[[354 212 217 116 190 115]]


In [ ]:
distances.flatten()

array([0.        , 0.3822636 , 0.42951941, 0.4490357 , 0.4510743 ,
       0.45461637], dtype=float32)

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Shutter Island (2010):

1: Inception (2010), with distance of 0.3822636008262634:
2: Inglourious Basterds (2009), with distance of 0.4295194149017334:
3: Dark Knight, The (2008), with distance of 0.4490357041358948:
4: Hangover, The (2009), with distance of 0.4510743021965027:
5: Dark Knight Rises, The (2012), with distance of 0.45461636781692505:
